In [39]:
import pandas as pd
import numpy as np

from ta import momentum, trend, volume
from binance import Client
from key import PUB_KEY, SEC_KEY

# Get Historical Data

In [40]:
# Connecting to Binance
client = Client(PUB_KEY, SEC_KEY)
# Get historical data
df = pd.DataFrame(client.get_historical_klines('BTCUSDT',
                             Client.KLINE_INTERVAL_1MINUTE,
                                      str(3000) + ' minutes ago UTC'))

# First 6 column represent Time and OHLCV values
df = df.iloc[:, :6]
df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']

# Convert first column into datetime
df['Time'] = pd.to_datetime(df['Time'], unit='ms')
for column in df.columns:
    if column != 'Time':
        df[column] = df[column].astype(float)

# Set Time as index
df.set_index('Time', inplace=True)

# Generate Technical Indicators

In [41]:
df

,Open,High,Low,Close,Volume
Time,,,,,
2024-07-09 08:45:00,57840.01,57850.00,57680.55,57687.73,76.02838
2024-07-09 08:46:00,57687.73,57768.00,57682.01,57721.98,37.85461
2024-07-09 08:47:00,57721.98,57820.22,57680.00,57817.59,51.47382
2024-07-09 08:48:00,57812.09,57829.12,57746.89,57807.43,57.78791
2024-07-09 08:49:00,57807.43,57830.00,57753.45,57753.46,24.21558
...,...,...,...,...,...
2024-07-11 10:40:00,58380.50,58388.00,58356.61,58356.62,7.26970
2024-07-11 10:41:00,58356.62,58394.44,58356.61,58384.01,7.23787
2024-07-11 10:42:00,58384.01,58384.01,58368.00,58368.01,5.80003


In [42]:
df['SMA50'] = df['Close'].rolling(50).mean()
df['SMA200'] = df['Close'].rolling(200).mean()
df['RSI'] = momentum.rsi(df['Close'])
df['MACD'] = trend.macd(df['Close'])
df['MACD_sig'] = trend.macd_signal(df['Close'])
df['MACD_diff'] = trend.macd_diff(df['Close'])
df['Momentum'] = (df['Close'] - df['Close'].shift(30))/df['Close'].shift(30)
df['ADX'] = trend.adx(df['High'], df['Low'], df['Close'])
df['-DI'] = trend.adx_neg(df['High'], df['Low'], df['Close'])
df['+DI'] = trend.adx_pos(df['High'], df['Low'], df['Close'])


# Generate Trading Signals

For each TAs, a buy, hold and sell signal will be generated. They will bear the weight of 1, 0 and -1 respectively

In [43]:
df.dropna(inplace=True)
df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,MACD_diff,Momentum,ADX,-DI,+DI
Time,,,,,,,,,,,,,,,
2024-07-09 12:04:00,57434.86,57458.16,57430.30,57434.11,8.99618,57453.4530,57511.01290,46.641274,-1.280928,-1.923344,0.642416,-0.000540,15.077382,26.807692,19.617511
2024-07-09 12:05:00,57434.11,57445.13,57378.97,57378.98,14.89405,57454.2326,57509.46915,38.102008,-6.807409,-2.900157,-3.907252,-0.002209,16.478288,34.404606,16.682468
2024-07-09 12:06:00,57378.98,57414.00,57378.96,57400.01,8.83026,57455.2728,57507.85930,42.431816,-9.382089,-4.196544,-5.185545,-0.001496,17.779351,31.701790,15.370810
2024-07-09 12:07:00,57400.00,57411.99,57400.00,57400.01,7.35136,57456.3130,57505.77140,42.431816,-11.292370,-5.615709,-5.676661,-0.001565,18.987482,30.809143,14.938005
2024-07-09 12:08:00,57400.00,57436.87,57400.00,57435.82,12.28140,57457.0408,57503.91335,49.419165,-9.803703,-6.453308,-3.350395,-0.000490,18.946005,28.181321,19.419528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-11 10:40:00,58380.50,58388.00,58356.61,58356.62,7.26970,58285.9816,58211.30520,59.315382,26.320750,20.265896,6.054854,0.002125,21.321090,17.628634,34.568779
2024-07-11 10:41:00,58356.62,58394.44,58356.61,58384.01,7.23787,58289.4218,58212.42520,63.168273,28.243228,21.861362,6.381865,0.003128,22.246332,16.248659,33.195332
2024-07-11 10:42:00,58384.01,58384.01,58368.00,58368.01,5.80003,58292.5118,58213.46520,59.616574,28.151229,23.119336,5.031893,0.002820,23.105486,15.688921,32.051809


In [47]:
#SMA Golden Cross
sma_buy_cond = (df['SMA50'] > df['SMA200']) & (df['SMA50'].shift(1) < df['SMA200'])
sma_sell_cond = (df['SMA50'] < df['SMA200']) & (df['SMA50'].shift(1) > df['SMA200'])
df['Signal_sma'] = np.where(sma_buy_cond, 1, np.where(sma_sell_cond, -1, 0))